<a href="https://colab.research.google.com/github/Sandeepan26/SR_ML_DL_RL_Repo/blob/main/neural_networks/resnet/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torchvision
import torch

*This is an implementation of Residual Network Architecture in Pytorch* </br>
[ResNet](https://arxiv.org/abs/1512.03385)

*A simple function to generate a neural network based on the input parameters*


In [3]:
def create_convolution(input_channels: int, output_channels: int, kernel_size: int, stride_val: int):
  network = [] #Creating an empty list to append the network and then produce it as the result of the function call
  network += torch.nn.Sequential(torch.nn.Conv2d(in_channels=input_channels, out_channels=output_channels, kernel_size=kernel_size, stride=stride_val, padding=(kernel_size//2)),
                                 torch.nn.BatchNorm2d(output_channels),
                                 torch.nn.ReLU())
  return (torch.nn.Sequential(torch.nn.Conv2d(in_channels=input_channels, out_channels=output_channels, kernel_size=kernel_size, stride=stride_val, padding=(kernel_size//2)),
                                 torch.nn.BatchNorm2d(output_channels),
                                 torch.nn.ReLU()))

In [4]:
# Using the function above, creating a simple 32-layer Residual Network
rsn_model = torch.nn.Sequential(create_convolution(3, 32, 7, 2),
                                torch.nn.Sequential(*[create_convolution(input_channels=32 if i == 0 else 64, output_channels= 64, kernel_size=3, stride_val = 2 if i == 0 else 1) for i in range (6)]),
                                torch.nn.Sequential(*[create_convolution(input_channels=64 if i == 0 else 128, output_channels=128, kernel_size=3, stride_val=2 if i == 0 else 1) for i in range(8)]),
                                torch.nn.Sequential(*[create_convolution(input_channels=128 if i  == 0 else 256, output_channels=256, kernel_size=3, stride_val=2 if i == 0 else 1) for i in range(12)]),
                                torch.nn.Sequential(*[create_convolution(input_channels=256 if i == 0 else 512, output_channels=512, kernel_size=3, stride_val=2 if i == 0 else 1) for i in range(6)]),
                                torch.nn.AvgPool2d(4),
                                torch.nn.Flatten(),
                                torch.nn.Linear(512,1000)
                                )


In [5]:
# Generating a simple class for the architecture

class arch_block(torch.nn.Module):    #This class inherits the torch.nn.Module

  def __init__(self, num_in_channels:int , num_intermediate_channels:int, identity_downsample = None, stride:int =1):
    super(arch_block, self).__init__()
    self.expansion = 4
    self.conv_1 = torch.nn.Conv2d(num_in_channels, num_intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False)
    self.bn_1 = torch.nn.BatchNorm2d(num_intermediate_channels)
    self.conv_2 = torch.nn.Conv2d(num_intermediate_channels, num_intermediate_channels, kernel_size = 3, stride=stride, padding = 1, bias = False)
    self.bn_2 = torch.nn.BatchNorm2d(num_intermediate_channels)
    self.conv_3 = torch.nn.Conv2d(num_intermediate_channels, num_intermediate_channels*self.expansion, stride=1, kernel_size=1, padding=0, bias=False)
    self.bn_3 = torch.nn.BatchNorm2d(num_intermediate_channels*self.expansion)
    self.relu = torch.nn.ReLU()
    self.identity_downsample= identity_downsample
    self.stride = stride

  def forward(self, x):
    self.identity = self.x.clone()
    self.x = self.conv_1(self.x)
    self.x = self.bn_1(self.x)
    self.conv_2(self.x)
    self.bn_2(self.x)
    self.conv_3(self.x)
    self.bn_3(self.x)

    if not isinstance(self.identity_downsample, None):
      self.identity = self.identity_downsample(self.identity)

    self.x += self.identity
    self.x = self.relu(self.x)

    return self.x



In [6]:
arch = arch_block(3,3)


*The following code block is a class for generating a ResNet block for 101 layers and beyond*

In [9]:
class ResNet(torch.nn.Module):

  def __init__(self, arch_block: arch_block, layers:list, num_image_channels:int, num_classes:int):
    super(ResNet, self).__init__()
    self.in_channels = 64
    self.conv_1 = torch.nn.Conv2d(in_channels = num_image_channels, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
    self.bn_1 = torch.nn.BatchNorm2d(64)
    self.relu = torch.nn.ReLU()
    self.maxpool2d = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    #layers for ResNet

    self.layer_1 = self._make_layer(arch_block, layers[0], intermediate_channels = 64, stride=1)
    self.layer_2 = self._make_layer(arch_block, layers[1], intermediate_channels = 128, stride=2)
    self.layer_3 = self._make_layer(arch_block, layers[2], intermediate_channels = 256, stride=2)
    self.layer_4 = self._make_layer(arch_block, layers[3], intermediate_channels = 512, stride=2)

    self.avgpool2d = torch.nn.AdaptiveAvgPool2d((1,1))
    self.fully_connected = torch.nn.Linear(512*4, num_classes)


  def forward(self, x):
    self.x = self.conv_1(self.x)
    self.x = self.bn_1(self.x)
    self.x = self.relu(self.x)
    self.x = self.maxpool2d(self.x)
    self.x = self.layer_1(self.x)
    self.x = self.layer_2(self.x)
    self.x = self.layer_3(self.x)
    self.x = self.layer_4(self.x)
    self.x = self.avgpool2d(self.x)
    self.x = self.x.reshape(self.x[0], -1)
    self.x = self.fully_connected(self.x)

    return self.x


  def _make_layer(self, arch_block:arch_block, num_residual_blocks: int, intermediate_channels, stride:int):

    self.identity_downsample = None
    self.layers = []

    if ((stride != 1) or (self.in_channels != intermediate_channels * 4)):
      self.identity_downsample = torch.nn.Sequential(
                                torch.nn.Conv2d(self.in_channels, intermediate_channels * 4, kernel_size=1, stride= stride, bias = False),
                                torch.nn.BatchNorm2d(intermediate_channels * 4))

    self.layers.append(arch_block(self.in_channels, intermediate_channels, identity_downsample=self.identity_downsample, stride=stride))

    self.in_channels = intermediate_channels * 4

    for i in range (num_residual_blocks - 1):
      self.layers.append(arch_block(self.in_channels, intermediate_channels))

    return torch.nn.Sequential(*self.layers)

In [14]:
def ResNet_50(image_channels:int =3, num_classes:int = 1000):
  return ResNet(arch_block, [3,4,6,3], image_channels, num_classes)

def ResNet_101(image_channels:int =3, num_classes:int = 1000):
  return ResNet(arch_block, [3,4,23,3], image_channels, num_classes)

def ResNet_152(image_channels:int =3, num_classes:int = 1000):
  return ResNet(arch_block, [3,8,36,3], image_channels, num_classes)


In [ ]:
ResNet_152() #Printing the ResNet block of 152 layers